# Algebra Dataset

In [21]:
import sys
import os
import pandas as pd
import json

from datasets import load_dataset, DatasetDict
from sklearn.model_selection import train_test_split

sys.path.append(os.path.abspath('../'))

## Loading and preprocessing the data

### Proof Net (?)

`hoskinson-center/proofnet` is in LEAN 3, we use the LEAN 4 translation used in the paper *DeepSeek Prover*

Not much discrete math, maybe subsets `Ireland-rosen` or `Rudin`?

In [15]:
df = pd.read_json('proofnet.jsonl', lines=True)

In [16]:
df['name'] = 'proofnet_' + df['name']
df = df.drop(columns=['goal'])

In [17]:
rosen_df = df[df['name'].str.contains('rosen')]
print(rosen_df)

Empty DataFrame
Columns: [name, split, informal_prefix, formal_statement, header]
Index: []


In [18]:
df.head()

,name,split,informal_prefix,formal_statement,header
0,proofnet_exercise_1_13a,valid,/-- Suppose that $f$ is holomorphic in an open...,theorem exercise_1_13a {f : ℂ → ℂ} (Ω : Set ℂ)...,import Mathlib\n\nopen Complex Filter Function...
1,proofnet_exercise_1_13b,test,/-- Suppose that $f$ is holomorphic in an open...,theorem exercise_1_13b {f : ℂ → ℂ} (Ω : Set ℂ)...,import Mathlib\n\nopen Complex Filter Function...
2,proofnet_exercise_1_13c,valid,/-- Suppose that $f$ is holomorphic in an open...,theorem exercise_1_13c {f : ℂ → ℂ} (Ω : Set ℂ)...,import Mathlib\n\nopen Complex Filter Function...
3,proofnet_exercise_1_19a,test,/-- Prove that the power series $\sum nz^n$ do...,theorem exercise_1_19a (z : ℂ) (hz : abs z = 1...,import Mathlib\n\nopen Complex Filter Function...
4,proofnet_exercise_1_19b,valid,/-- Prove that the power series $\sum zn/n^2$ ...,theorem exercise_1_19b (z : ℂ) (hz : abs z = 1...,import Mathlib\n\nopen Complex Filter Function...


In [19]:
print(putnam_NT_putnam_NT_df.iloc[0].informal_prefix)

/-- Suppose that $f$ is holomorphic in an open set $\Omega$. Prove that if $\text{Re}(f)$ is constant, then $f$ is constant.-/



In [20]:
print(df.iloc[0].formal_statement)

theorem exercise_1_13a {f : ℂ → ℂ} (Ω : Set ℂ) (a b : Ω) (h : IsOpen Ω)
  (hf : DifferentiableOn ℂ f Ω) (hc : ∃ (c : ℝ), ∀ z ∈ Ω, (f z).re = c) :
  f a = f b :=


In [21]:
topics = set()
for i in range(len(df)):
    topics.update(df.iloc[i]['header'].split('\n'))

for topic in topics:
    print(topic)


open scoped BigOperators
| 0 => sqrt 2
  generateFrom {S : Set X | ∃ a b, a < b ∧ S = Ico a b}
def countably_compact (X : Type*) [TopologicalSpace X] :=
def rational (x : ℝ) := x ∈ range ((↑) : ℚ → ℝ)
  ∀ U : Set X, Infinite U → ∃ x ∈ U, ClusterPt x (𝓟 U)
noncomputable section
def lower_limit_topology (X : Type) [Preorder X] :=
  (∀ i, IsOpen (U i)) ∧ ((univ : Set X) ⊆ ⋃ i, U i) →
  (∃ t : Finset ℕ, (univ : Set X) ⊆ ⋃ i ∈ t, U i)
open Topology Filter Real Complex TopologicalSpace Finset
| (n + 1) => sqrt (2 + sqrt (f n))
open scoped BigOperators Topology
open Fintype Set Real Ideal Polynomial
  ∀ U : ℕ → Set X,
  univ ∈ T ∧
import Mathlib
  ({S : Set ℝ | ∃ a b, a < b ∧ S = Ioo a b} ∪ {S : Set ℝ | ∃ a b, a < b ∧ S = Ioo a b \ K})
open Complex Filter Function Metric Finset
def K : Set ℝ := {r | ∃ n : ℕ, r = 1 / n}
noncomputable def f : ℕ → ℝ
def g (n : ℕ) : ℝ := sqrt (n + 1) - sqrt n
set_option checkBinderAnnotations false
open Filter Real Function
open RingHom
--center of (G × H) equiv

In [23]:
raise Exception('stop')

Exception: stop

### Putnam Bench (273)

- 273 rows for `{'abstract_algebra', 'algebra'}`

In [22]:
from datasets import load_dataset
ds = load_dataset("amitayusht/PutnamBench")

Repo card metadata block was not found. Setting CardData to empty.


In [30]:
df = pd.DataFrame(ds['train'])

In [31]:
df = df.drop(columns=['coq_statement', 'isabelle_statement', 'informal_solution'])
df = df.dropna()
df = df.rename(columns={'lean4_statement': 'formal_statement'})

df['header'] = df['formal_statement'].apply(lambda x: x.split('theorem', 1)[0])
df['formal_statement'] = df['formal_statement'].apply(lambda x: 'theorem' + x.split('theorem', 1)[1] if 'theorem' in x else x)

In [32]:
all_tags = set()

for tags in df.tags.unique():
    tags = tags.replace('[', '').replace(']', '').replace('\'', '')
    tags = tags
    tag_list = tags.split(',')
    for tag in tag_list:
        all_tags.add(tag.strip())

print(all_tags)

{'algebra', 'analysis', 'number_theory', 'linear_algebra', 'abstract_algebra', 'set_theory', 'probability', 'combinatorics', 'geometry'}


In [33]:
wanted_tags = ['algebra', 'abstract_algebra']
putnam_df = df[df.tags.str.contains('|'.join(wanted_tags))]

putnam_NT_df = df[df.tags.str.contains('algebra')]


In [34]:
# shuffle df
putnam_NT_df = putnam_NT_df.sample(frac=1).reset_index(drop=True)
putnam_NT_df['split'] = ['valid' if x < len(putnam_NT_df) / 2 else 'test' for x in range(len(putnam_NT_df))]

In [35]:
len(putnam_NT_df)

273

In [36]:
putnam_NT_df.head()

,name,formal_statement,informal_statement,tags,header,split
0,putnam_2010_b4,theorem putnam_2010_b4\n(p q : Polynomial ℝ)\n...,Find all pairs of polynomials $p(x)$ and $q(x)...,['algebra'],abbrev putnam_2010_b4_solution : Set (Polynomi...,valid
1,putnam_1998_a4,theorem putnam_1998_a4\n(A : ℕ → ℕ)\n(hA1 : A ...,"Let $A_1=0$ and $A_2=1$. For $n>2$, the number...",['algebra'],abbrev putnam_1998_a4_solution : Set ℕ := sorr...,valid
2,putnam_2008_a4,theorem putnam_2008_a4\n(f : ℝ → ℝ)\n(hf : f =...,Define $f : \mathbb{R} \to \mathbb{R} by $f(x)...,['algebra'],abbrev putnam_2008_a4_solution : Prop := sorry...,valid
3,putnam_1992_b6,theorem putnam_1992_b6\n(n : ℕ)\n(npos : n > 0...,Let $M$ be a set of real $n \times n$ matrices...,['linear_algebra'],,valid
4,putnam_1980_b5,theorem putnam_1980_b5\n(T : Set ℝ := Icc 0 1)...,"A function $f$ is convex on $[0, 1]$ if and on...","['analysis', 'algebra']",abbrev putnam_1980_b5_solution : ℝ → Prop := s...,valid


#### Look at the data:

In [37]:
print(putnam_NT_df.iloc[0].informal_statement)

Find all pairs of polynomials $p(x)$ and $q(x)$ with real coefficients for which $p(x)q(x+1)-p(x+1)q(x)=1$.


In [38]:
print(putnam_NT_df.iloc[0].formal_statement)

theorem putnam_2010_b4
(p q : Polynomial ℝ)
: (∀ x : ℝ, p.eval x * q.eval (x + 1) - p.eval (x + 1) * q.eval x = 1) ↔ (p, q) ∈ putnam_2010_b4_solution :=
sorry


In [39]:
NT_tag = putnam_NT_df.iloc[3].tags
print(NT_tag)

['linear_algebra']


In [56]:
raise Exception('stop')

Exception: stop

### Mini F2F (177)

Keep only rows about `number_theory`

In [45]:
df = pd.read_json('minif2f.jsonl', lines=True)

In [46]:
df['name'] = 'f2f_' + df['name']
df['tags'] = "['algebra']"

df = df.drop(columns=['goal'])
df = df.rename(columns={'informal_prefix': 'informal_statement'})
df = df[df['name'].str.contains('algebra')]

df = df[['name', 'formal_statement', 'informal_statement', 'tags', 'header',
       'split']]

In [47]:
df['informal_statement'] = df['informal_statement'].str.replace(r'^/-- ', '', regex=True).str.replace(r'-/\n$', '', regex=True)

In [48]:
df.head()

,name,formal_statement,informal_statement,tags,header,split
3,f2f_mathd_algebra_182,theorem mathd_algebra_182 (y : ℂ) : 7 * (3 * y...,Expand the following expression: $7(3y+2)$ Sho...,['algebra'],import Mathlib\nimport Aesop\n\nset_option max...,valid
6,f2f_mathd_algebra_116,theorem mathd_algebra_116 (k x : ℝ) (h₀ : x = ...,For what real value of $k$ is $\frac{13-\sqrt{...,['algebra'],import Mathlib\nimport Aesop\n\nset_option max...,valid
11,f2f_mathd_algebra_13,theorem mathd_algebra_13 (a b : ℝ)\n (h₀ : ∀ ...,Find $A$ and $B$ such that\n\[\frac{4x}{x^2-8x...,['algebra'],import Mathlib\nimport Aesop\n\nset_option max...,valid
18,f2f_mathd_algebra_462,theorem mathd_algebra_462 : ((1 : ℚ) / 2 + 1 /...,Evaluate: $\left( \frac{1}{2} + \frac{1}{3} \r...,['algebra'],import Mathlib\nimport Aesop\n\nset_option max...,valid
29,f2f_mathd_algebra_48,theorem mathd_algebra_48 (q e : ℂ) (h₀ : q = 9...,Simplify $(9-4i)-(-3-4i)$. Show that it is 12.,['algebra'],import Mathlib\nimport Aesop\n\nset_option max...,valid


In [49]:
len(df)

177

#### Look at the data:

In [50]:
print(df.iloc[0].informal_statement)

Expand the following expression: $7(3y+2)$ Show that it is 21y+14.


In [51]:
print(df.iloc[0].header)

import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat




In [52]:
print(df.iloc[0].formal_statement)

theorem mathd_algebra_182 (y : ℂ) : 7 * (3 * y + 2) = 21 * y + 14 := by



## Aggregating the data (450)

In [53]:
print(df.columns)
print(putnam_NT_df.columns)

Index(['name', 'formal_statement', 'informal_statement', 'tags', 'header',
       'split'],
      dtype='object')
Index(['name', 'formal_statement', 'informal_statement', 'tags', 'header',
       'split'],
      dtype='object')


In [54]:
final_df = pd.concat([df, putnam_NT_df], ignore_index=True)

In [55]:
final_df.head()

,name,formal_statement,informal_statement,tags,header,split
0,f2f_mathd_algebra_182,theorem mathd_algebra_182 (y : ℂ) : 7 * (3 * y...,Expand the following expression: $7(3y+2)$ Sho...,['algebra'],import Mathlib\nimport Aesop\n\nset_option max...,valid
1,f2f_mathd_algebra_116,theorem mathd_algebra_116 (k x : ℝ) (h₀ : x = ...,For what real value of $k$ is $\frac{13-\sqrt{...,['algebra'],import Mathlib\nimport Aesop\n\nset_option max...,valid
2,f2f_mathd_algebra_13,theorem mathd_algebra_13 (a b : ℝ)\n (h₀ : ∀ ...,Find $A$ and $B$ such that\n\[\frac{4x}{x^2-8x...,['algebra'],import Mathlib\nimport Aesop\n\nset_option max...,valid
3,f2f_mathd_algebra_462,theorem mathd_algebra_462 : ((1 : ℚ) / 2 + 1 /...,Evaluate: $\left( \frac{1}{2} + \frac{1}{3} \r...,['algebra'],import Mathlib\nimport Aesop\n\nset_option max...,valid
4,f2f_mathd_algebra_48,theorem mathd_algebra_48 (q e : ℂ) (h₀ : q = 9...,Simplify $(9-4i)-(-3-4i)$. Show that it is 12.,['algebra'],import Mathlib\nimport Aesop\n\nset_option max...,valid


In [56]:
len(final_df)

450

## Export

In [57]:
final_df[final_df['split'] == 'test'].to_json('ALG_test.jsonl', orient='records', lines=True)
final_df[final_df['split'] == 'valid'].to_json('ALG_validation.jsonl', orient='records', lines=True)